# Вступ до ймовірності та статистики
У цій записній книжці ми пограємося з деякими концепціями, які ми раніше обговорювали. Багато концепцій з ймовірності та статистики добре представлені у основних бібліотеках для обробки даних у Python, таких як `numpy` та `pandas`.


In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

## Випадкові величини та розподіли
Почнемо з вибірки з 30 значень з рівномірного розподілу від 0 до 9. Також обчислимо середнє та дисперсію.


In [ ]:
sample = [ random.randint(0,10) for _ in range(30) ]
print(f"Sample: {sample}")
print(f"Mean = {np.mean(sample)}")
print(f"Variance = {np.var(sample)}")

Щоб візуально оцінити, скільки різних значень є в вибірці, ми можемо побудувати **гістограму**:


In [ ]:
plt.hist(sample)
plt.show()

## Аналіз реальних даних

Середнє значення та дисперсія дуже важливі при аналізі реальних даних. Давайте завантажимо дані про бейсболістів із [SOCR MLB Height/Weight Data](http://wiki.stat.ucla.edu/socr/index.php/SOCR_Data_MLB_HeightsWeights)


In [ ]:
df = pd.read_csv("../../data/SOCR_MLB.tsv",sep='\t', header=None, names=['Name','Team','Role','Weight','Height','Age'])
df


> Ми використовуємо пакет під назвою [**Pandas**](https://pandas.pydata.org/) для аналізу даних. Про Pandas та роботу з даними в Python ми поговоримо пізніше у цьому курсі.

Давайте обчислимо середні значення для віку, зросту та ваги:


In [ ]:
df[['Age','Height','Weight']].mean()

Тепер зосередимося на зрості та обчислимо стандартне відхилення і дисперсію:


In [ ]:
print(list(df['Height'])[:20])

In [ ]:
mean = df['Height'].mean()
var = df['Height'].var()
std = df['Height'].std()
print(f"Mean = {mean}\nVariance = {var}\nStandard Deviation = {std}")

Окрім середнього значення, має сенс розглянути медіану та квартилі. Їх можна візуалізувати за допомогою **коробчастої діаграми**:


In [ ]:
plt.figure(figsize=(10,2))
plt.boxplot(df['Height'].ffill(), vert=False, showmeans=True)
plt.grid(color='gray', linestyle='dotted')
plt.tight_layout()
plt.show()

Ми також можемо створювати коробкові діаграми для підмножин нашого набору даних, наприклад, згрупованих за роллю гравця.


In [ ]:
df.boxplot(column='Height', by='Role', figsize=(10,8))
plt.xticks(rotation='vertical')
plt.tight_layout()
plt.show()

> **Примітка**: Ця діаграма свідчить, що в середньому зріст перших бейссменів вищий за зріст других бейссменів. Пізніше ми дізнаємося, як формально протестувати цю гіпотезу та як показати, що наші дані статистично значущі для цього.  

Вік, зріст і вага — це всі неперервні випадкові змінні. Як ви гадаєте, який у них розподіл? Хороший спосіб дізнатись це — побудувати гістограму значень: 


In [ ]:
df['Weight'].hist(bins=15, figsize=(10,6))
plt.suptitle('Weight distribution of MLB Players')
plt.xlabel('Weight')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

## Normal Distribution

Давайте створимо штучну вибірку ваг, яка відповідає нормальному розподілу з таким самим середнім і дисперсією, як і наші реальні дані:


In [ ]:
generated = np.random.normal(mean, std, 1000)
generated[:20]

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(generated, bins=15)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(np.random.normal(0,1,50000), bins=300)
plt.tight_layout()
plt.show()

Оскільки більшість значень у реальному житті розподілені нормально, ми не повинні використовувати рівномірний генератор випадкових чисел для генерації вибіркових даних. Ось що відбувається, якщо ми спробуємо згенерувати ваги з рівномірним розподілом (згенерованим за допомогою `np.random.rand`):


In [ ]:
wrong_sample = np.random.rand(1000)*2*std+mean-std
plt.figure(figsize=(10,6))
plt.hist(wrong_sample)
plt.tight_layout()
plt.show()

## Довіpчі інтервали

Тепер обчислимо довірчі інтервали для ваги та зросту бейсбольних гравців. Ми використаємо код [з цієї дискусії на stackoverflow](https://stackoverflow.com/questions/15033511/compute-a-confidence-interval-from-sample-data):


In [ ]:
import scipy.stats

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, h

for p in [0.85, 0.9, 0.95]:
    m, h = mean_confidence_interval(df['Weight'].fillna(method='pad'),p)
    print(f"p={p:.2f}, mean = {m:.2f} ± {h:.2f}")

## Перевірка гіпотез

Давайте дослідимо різні ролі в нашому наборі даних бейсбольних гравців:


In [ ]:
df.groupby('Role').agg({ 'Weight' : 'mean', 'Height' : 'mean', 'Age' : 'count'}).rename(columns={ 'Age' : 'Count'})

Давайте перевіримо гіпотезу, що перші бейсмені вищі за других бейсменів. Найпростіший спосіб зробити це — перевірити довірчі інтервали:


In [ ]:
for p in [0.85,0.9,0.95]:
    m1, h1 = mean_confidence_interval(df.loc[df['Role']=='First_Baseman',['Height']],p)
    m2, h2 = mean_confidence_interval(df.loc[df['Role']=='Second_Baseman',['Height']],p)
    print(f'Conf={p:.2f}, 1st basemen height: {m1-h1[0]:.2f}..{m1+h1[0]:.2f}, 2nd basemen height: {m2-h2[0]:.2f}..{m2+h2[0]:.2f}')

Ми бачимо, що інтервали не перекриваються.

Статистично більш коректний спосіб підтвердити гіпотезу — використати **t-тест Стьюдента**:


In [ ]:
from scipy.stats import ttest_ind

tval, pval = ttest_ind(df.loc[df['Role']=='First_Baseman',['Height']], df.loc[df['Role']=='Second_Baseman',['Height']],equal_var=False)
print(f"T-value = {tval[0]:.2f}\nP-value: {pval[0]}")

Два значення, які повертає функція `ttest_ind`, є:
* p-значення можна розглядати як ймовірність того, що два розподіли мають однакове середнє. У нашому випадку воно дуже низьке, що означає, що є сильні докази на користь того, що першопластуни вищі.
* t-значення — це проміжне значення нормалізованої різниці середніх, яке використовується в t-тесті, і його порівнюють з пороговим значенням для заданого рівня довіри.


## Моделювання нормального розподілу за допомогою центральної граничної теореми

Псевдовипадковий генератор у Python призначений для надання нам рівномірного розподілу. Якщо ми хочемо створити генератор для нормального розподілу, ми можемо використати центральну граничну теорему. Щоб отримати нормально розподілене значення, ми просто обчислимо середнє значення вибірки, згенерованої рівномірно.


In [ ]:
def normal_random(sample_size=100):
    sample = [random.uniform(0,1) for _ in range(sample_size) ]
    return sum(sample)/sample_size

sample = [normal_random() for _ in range(100)]
plt.figure(figsize=(10,6))
plt.hist(sample)
plt.tight_layout()
plt.show()

## Кореляція та Зла Бейсбольна Корпорація

Кореляція дозволяє нам знаходити зв’язки між послідовностями даних. У нашому іграшковому прикладі уявімо, що існує зла бейсбольна корпорація, яка платить своїм гравцям відповідно до їхнього зросту — чим вищий гравець, тим більше він/вона отримує грошей. Припустимо, що базова зарплата становить 1000 доларів, а додатковий бонус коливається від 0 до 100 доларів, залежно від зросту. Ми візьмемо реальних гравців з MLB і порахуємо їхні уявні зарплати:


In [ ]:
heights = df['Height'].fillna(method='pad')
salaries = 1000+(heights-heights.min())/(heights.max()-heights.mean())*100
print(list(zip(heights, salaries))[:10])

Давайте тепер обчислимо коваріацію та кореляцію цих послідовностей. `np.cov` дасть нам так звану **матриду коваріації**, яка є розширенням коваріації на кілька змінних. Елемент $M_{ij}$ матриці коваріації $M$ є коваріацією між вхідними змінними $X_i$ та $X_j$, а діагональні значення $M_{ii}$ — це дисперсія $X_{i}$. Аналогічно, `np.corrcoef` дасть нам **матриду кореляції**.


In [ ]:
print(f"Covariance matrix:\n{np.cov(heights, salaries)}")
print(f"Covariance = {np.cov(heights, salaries)[0,1]}")
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

Кореляція, рівна 1, означає, що існує сильний **лінійний зв’язок** між двома змінними. Ми можемо візуально побачити лінійний зв’язок, побудувавши графік одного значення проти іншого:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights,salaries)
plt.tight_layout()
plt.show()

Давайте подивимося, що станеться, якщо залежність не буде лінійною. Припустимо, наша корпорація вирішила приховати очевидну лінійну залежність між зростом і зарплатою, і ввела деяку нелінійність у формулу, наприклад `sin`:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

У цьому випадку кореляція трохи менша, але все ще досить висока. Тепер, щоб зробити зв’язок ще менш очевидним, ми можемо додати трохи додаткової випадковості, додавши до зарплати випадкову змінну. Давайте подивимося, що станеся:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100+np.random.random(size=len(heights))*20-10
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights, salaries)
plt.tight_layout()
plt.show()

> Чи можете ви здогадатися, чому крапки вирівнюються у вертикальні лінії саме так?

Ми спостерігали кореляцію між штучно створеним поняттям, таким як зарплата, та спостережуваною змінною *зріст*. Давайте також перевіримо, чи корелюють між собою дві спостережувані змінні, такі як зріст і вага:


In [ ]:
np.corrcoef(df['Height'].ffill(),df['Weight'])

На жаль, ми не отримали жодних результатів — лише деякі дивні значення `nan`. Це пов’язано з тим, що деякі значення в нашому ряді невизначені, представлені як `nan`, що викликає також невизначений результат операції. Розглядаючи матрицю, ми бачимо, що проблемним стовпцем є `Weight`, оскільки обчислювалася самокореляція між значеннями `Height`.

> Цей приклад демонструє важливість **підготовки даних** та **очищення**. Без належних даних ми не можемо нічого обчислити.

Давайте використаємо метод `fillna` для заповнення пропущених значень і обчислимо кореляцію:


In [ ]:
np.corrcoef(df['Height'].fillna(method='pad'), df['Weight'])

Дійсно існує кореляція, але не така сильна, як у нашому штучному прикладі. Насправді, якщо подивитися на діаграму розсіяння одного значення проти іншого, зв’язок буде значно менш очевидним:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(df['Weight'],df['Height'])
plt.xlabel('Weight')
plt.ylabel('Height')
plt.tight_layout()
plt.show()

## Висновок

У цьому нотатнику ми навчились виконувати базові операції з даними для обчислення статистичних функцій. Тепер ми знаємо, як використовувати надійний апарат математики та статистики, щоб доводити деякі гіпотези, а також як обчислювати довірчі інтервали для довільних змінних на основі вибірки даних.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Відмова від відповідальності**:
Цей документ було перекладено за допомогою сервісу автоматичного перекладу [Co-op Translator](https://github.com/Azure/co-op-translator). Хоча ми прагнемо до точності, зверніть увагу, що автоматичні переклади можуть містити помилки або неточності. Оригінальний документ його рідною мовою слід вважати авторитетним джерелом. Для критично важливої інформації рекомендується звертатися до професійного людського перекладу. Ми не несемо відповідальності за будь-які непорозуміння або неправильне тлумачення, що виникли внаслідок використання цього перекладу.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
